## General functionality of MC Dropout

In [1]:
%load_ext autoreload

In [2]:
import os, sys, importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

BASE_PATH = os.path.join(os.getcwd(), "..", "..")
MODULE_PATH = os.path.join(BASE_PATH, "modules")
DATASET_PATH = os.path.join(BASE_PATH, "datasets")

sys.path.append(MODULE_PATH)

from bayesian import McDropout
from data import BenchmarkData, DataSetType
from models import setup_growth, FcholletCNN

In [3]:
setup_growth()

1 Physical GPU's,  1 Logical GPU's


In [4]:
benchmark_data = BenchmarkData(DataSetType.MNIST, os.path.join(DATASET_PATH, "mnist"), classes=4, dtype=np.float32)

In [5]:
benchmark_data.inputs.shape

(28911, 28, 28, 1)

In [6]:
inputs = benchmark_data.inputs
targets = benchmark_data.targets

selector = np.isin(targets, np.unique(targets)[:-1])
new_targets = targets[selector]
new_inputs = inputs[selector]

ood_selector = np.logical_not(selector)
ood_targets = targets[ood_selector]
ood_inputs = inputs[ood_selector]

In [7]:
x_train, x_test, y_train, y_test = train_test_split(new_inputs, new_targets)

In [8]:
%autoreload 2
model = FcholletCNN(output=3)
model.build(input_shape=inputs.shape)
model.summary()

Model: "Fchollet-CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Convolutions (Sequential)    multiple                  18816     
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
Linear (Sequential)          multiple                  1606147   
Total params: 1,624,963
Trainable params: 1,624,963
Non-trainable params: 0
_________________________________________________________________


In [9]:
mc_dropout = McDropout(model)
mc_dropout.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

In [11]:
mc_dropout.fit(x_train, y_train, epochs=10, batch_size=60)

Epoch 1/10
273/273 [==============================] - 1s 5ms/step - loss: 0.6550 - accuracy: 0.8964
Epoch 2/10
273/273 [==============================] - 1s 5ms/step - loss: 0.6042 - accuracy: 0.9473
Epoch 3/10
273/273 [==============================] - 1s 4ms/step - loss: 0.5898 - accuracy: 0.9616
Epoch 4/10
273/273 [==============================] - 1s 4ms/step - loss: 0.6162 - accuracy: 0.9351
Epoch 5/10
273/273 [==============================] - 1s 4ms/step - loss: 0.6111 - accuracy: 0.9403
Epoch 6/10
273/273 [==============================] - 1s 4ms/step - loss: 0.6006 - accuracy: 0.9509
Epoch 7/10
273/273 [==============================] - 1s 4ms/step - loss: 0.5965 - accuracy: 0.9550
Epoch 8/10
273/273 [==============================] - 1s 4ms/step - loss: 0.5901 - accuracy: 0.9614
Epoch 9/10
273/273 [==============================] - 1s 5ms/step - loss: 0.6059 - accuracy: 0.9456
Epoch 10/10
273/273 [==============================] - 1s 4ms/step - loss: 0.6008 - accuracy: 0.9506

In [19]:
mc_dropout._model.evaluate(x_test, y_test)

171/171 [==============================] - 4s 21ms/step - loss: 0.5700 - accuracy: 0.9814


[0.5700008869171143, 0.9814440608024597]

In [27]:
%autoreload 2
mc_dropout.evaluate(x_test, y_test, batch_size=60, runs=1)

(5443, 3)
(5443, 3, 2)
[[0 1]
 [0 1]
 [0 1]
 ...
 [0 1]
 [0 1]
 [0 1]]
[0 0 0 ... 0 0 0]
False


[0.5907753109931946, 0.0]

In [26]:
%autoreload 2
NUM_DATAPOINTS = 20

result = mc_dropout.predict(x_test[:NUM_DATAPOINTS].astype(np.float32), runs=1000)
result.shape

(20, 1000, 3)

In [13]:
class_probs = np.mean(result, axis=1)
class_probs

array([[0.251     , 0.37      , 0.379     ],
       [0.25      , 0.36000278, 0.38999722],
       [0.252     , 0.365     , 0.383     ],
       [0.252     , 0.363     , 0.385     ],
       [0.252     , 0.369     , 0.379     ],
       [0.251     , 0.36399957, 0.38500043],
       [0.25      , 0.37      , 0.38      ],
       [0.25      , 0.369     , 0.381     ],
       [0.25      , 0.36500003, 0.38499997],
       [0.253     , 0.359     , 0.388     ],
       [0.251     , 0.35599983, 0.39300017],
       [0.251     , 0.372     , 0.377     ],
       [0.25      , 0.36199998, 0.38800002],
       [0.251     , 0.365     , 0.384     ],
       [0.251     , 0.36758771, 0.38141229],
       [0.251     , 0.365     , 0.384     ],
       [0.252     , 0.362     , 0.38600001],
       [0.25      , 0.35995125, 0.39004875],
       [0.25197614, 0.363     , 0.38502386],
       [0.249     , 0.371     , 0.38      ]])

In [14]:
label_prediction = np.argmax(class_probs, axis=1)
label_prediction

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [15]:
acc = np.mean(label_prediction == y_test[:NUM_DATAPOINTS])
acc

0.35